<a href="https://colab.research.google.com/github/caioitalo/soulcode/blob/main/Pr%C3%A1tica_DB_transa%C3%A7%C3%B5es_de_cart%C3%A3o_india.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [78]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import numpy as np

In [79]:
df = pd.read_csv('https://storage.googleapis.com/datalake-soulcode-ed7/arquivos%20soulcode/Credit%20card%20transactions%20-%20India%20-%20Simple.csv')

In [80]:
df

,index,City,Date,Card Type,Exp Type,Gender,Amount
0,0,"Delhi, India",29-Oct-14,Gold,Bills,F,82475
1,1,"Greater Mumbai, India",22-Aug-14,Platinum,Bills,F,32555
2,2,"Bengaluru, India",27-Aug-14,Silver,Bills,F,101738
3,3,"Greater Mumbai, India",12-Apr-14,Signature,Bills,F,123424
4,4,"Bengaluru, India",5-May-15,Gold,Bills,F,171574
...,...,...,...,...,...,...,...
26047,26047,"Kolkata, India",22-Jun-14,Silver,Travel,F,128191
26048,26048,"Pune, India",3-Aug-14,Signature,Travel,M,246316
26049,26049,"Hyderabad, India",16-Jan-15,Silver,Travel,M,265019
26050,26050,"Kanpur, India",14-Sep-14,Silver,Travel,M,88174


In [81]:
df.dtypes

index         int64
City         object
Date         object
Card Type    object
Exp Type     object
Gender       object
Amount        int64
dtype: object

In [82]:
df.isna().sum()

index        0
City         0
Date         0
Card Type    0
Exp Type     0
Gender       0
Amount       0
dtype: int64

In [83]:
df['Exp Type'].unique()

array(['Bills', 'Food', 'Entertainment', 'Grocery', 'Fuel', 'Travel'],
      dtype=object)

In [84]:
df = df.rename(columns={'index':'id_compra','City':'cidade','Date':'data','Card Type':'tipo_cartao','Exp Type':'tipo_compra','Gender':'sexo','Amount':'montante'})

In [85]:
spark = (SparkSession.builder
                     .master('local')
                     .appName('spark_app')
                     .config('spark.ui.port', '4050')
                     .getOrCreate()
                     )

In [86]:
dfsp = spark.createDataFrame(df)

In [87]:
dfsp.show()

+---------+--------------------+---------+-----------+-----------+----+--------+
|id_compra|              cidade|     data|tipo_cartao|tipo_compra|sexo|montante|
+---------+--------------------+---------+-----------+-----------+----+--------+
|        0|        Delhi, India|29-Oct-14|       Gold|      Bills|   F|   82475|
|        1|Greater Mumbai, I...|22-Aug-14|   Platinum|      Bills|   F|   32555|
|        2|    Bengaluru, India|27-Aug-14|     Silver|      Bills|   F|  101738|
|        3|Greater Mumbai, I...|12-Apr-14|  Signature|      Bills|   F|  123424|
|        4|    Bengaluru, India| 5-May-15|       Gold|      Bills|   F|  171574|
|        5|        Delhi, India| 8-Sep-14|     Silver|      Bills|   F|  100036|
|        6|        Delhi, India|24-Feb-15|       Gold|      Bills|   F|  143250|
|        7|Greater Mumbai, I...|26-Jun-14|   Platinum|      Bills|   F|  150980|
|        8|        Delhi, India|28-Mar-14|     Silver|      Bills|   F|  192247|
|        9|        Delhi, In

In [88]:
dfsp = dfsp.withColumn('cidade_real',F.split(F.col('cidade'),',').getItem(0)).withColumn('pais',F.split(F.col('cidade'),',').getItem(1))

In [89]:
dfsp = dfsp.drop(F.col('cidade'))

In [90]:
df2 = dfsp

In [91]:
df2 = df2.drop('id_compra')

In [92]:
df2 = df2.dropDuplicates()

In [93]:
df2.show(5)

+---------+-----------+-----------+----+--------+--------------+------+
|     data|tipo_cartao|tipo_compra|sexo|montante|   cidade_real|  pais|
+---------+-----------+-----------+----+--------+--------------+------+
| 3-Mar-15|   Platinum|      Bills|   F|  738108|Greater Mumbai| India|
|26-Mar-14|       Gold|      Bills|   F|  234200|         Delhi| India|
| 6-May-14|     Silver|       Food|   F|  138342|     Bengaluru| India|
|24-Feb-14|       Gold|       Food|   F|  236980|Greater Mumbai| India|
|24-Dec-14|     Silver|       Food|   F|  168546|     Bengaluru| India|
+---------+-----------+-----------+----+--------+--------------+------+
only showing top 5 rows



In [94]:
df2 = df2.withColumnRenamed('cidade_real','cidade')

In [95]:
df2.groupBy('pais').count().show()

+------+-----+
|  pais|count|
+------+-----+
| India|26052|
+------+-----+



In [96]:
df2 = df2.drop('pais')

In [97]:
df2 = (df2.withColumn('data',F.regexp_replace('data','Jan','01'))
         .withColumn('data',F.regexp_replace('data','Feb','02'))
         .withColumn('data',F.regexp_replace('data','Mar','03'))
         .withColumn('data',F.regexp_replace('data','Apr','04')) 
         .withColumn('data',F.regexp_replace('data','May','05'))
         .withColumn('data',F.regexp_replace('data','Jun','06')) 
         .withColumn('data',F.regexp_replace('data','Jul','07'))
         .withColumn('data',F.regexp_replace('data','Aug','08')) 
         .withColumn('data',F.regexp_replace('data','Sep','09'))
         .withColumn('data',F.regexp_replace('data','Oct','10')) 
         .withColumn('data',F.regexp_replace('data','Nov','11'))
         .withColumn('data',F.regexp_replace('data','Dec','12'))
)

In [98]:
df2.show(5)

+--------+-----------+-----------+----+--------+--------------+
|    data|tipo_cartao|tipo_compra|sexo|montante|        cidade|
+--------+-----------+-----------+----+--------+--------------+
| 3-03-15|   Platinum|      Bills|   F|  738108|Greater Mumbai|
|26-03-14|       Gold|      Bills|   F|  234200|         Delhi|
| 6-05-14|     Silver|       Food|   F|  138342|     Bengaluru|
|24-02-14|       Gold|       Food|   F|  236980|Greater Mumbai|
|24-12-14|     Silver|       Food|   F|  168546|     Bengaluru|
+--------+-----------+-----------+----+--------+--------------+
only showing top 5 rows



In [99]:
df2 = df2.withColumn('dia', F.split(F.col('data'),'-').getItem(0))\
         .withColumn('mes', F.split(F.col('data'),'-').getItem(1))\
         .withColumn('ano', F.split(F.col('data'),'-').getItem(2))
df2.show(5)

+--------+-----------+-----------+----+--------+--------------+---+---+---+
|    data|tipo_cartao|tipo_compra|sexo|montante|        cidade|dia|mes|ano|
+--------+-----------+-----------+----+--------+--------------+---+---+---+
| 3-03-15|   Platinum|      Bills|   F|  738108|Greater Mumbai|  3| 03| 15|
|26-03-14|       Gold|      Bills|   F|  234200|         Delhi| 26| 03| 14|
| 6-05-14|     Silver|       Food|   F|  138342|     Bengaluru|  6| 05| 14|
|24-02-14|       Gold|       Food|   F|  236980|Greater Mumbai| 24| 02| 14|
|24-12-14|     Silver|       Food|   F|  168546|     Bengaluru| 24| 12| 14|
+--------+-----------+-----------+----+--------+--------------+---+---+---+
only showing top 5 rows



In [100]:
df2.groupBy('ano').count().show()

+---+-----+
|ano|count|
+---+-----+
| 15| 6334|
| 13| 3927|
| 14|15791|
+---+-----+



In [101]:
df2 = (df2.withColumn('dia', F.when(F.col('dia')=='1','01')
                              .when(F.col('dia')=='2','02')
                              .when(F.col('dia')=='3','03')
                              .when(F.col('dia')=='4','04')
                              .when(F.col('dia')=='5','05')
                              .when(F.col('dia')=='6','06')
                              .when(F.col('dia')=='7','07')
                              .when(F.col('dia')=='8','08')
                              .when(F.col('dia')=='9','09')
                              .otherwise(F.col('dia')))
      )

df2 = (df2.withColumn('ano', F.when(F.col('ano')=='13','2013')
                              .when(F.col('ano')=='14','2014')
                              .otherwise('2015'))
      )

In [102]:
df2 = df2.withColumn('data',F.concat(F.col('ano'),F.lit('-'),F.col('mes'),F.lit('-'),F.col('dia')))
df2 = df2.drop('dia','mes','ano')
df2 = df2.withColumn('data',F.to_date(F.col('data'),'yyyy-MM-dd'))
df2.show(5)

+----------+-----------+-----------+----+--------+--------------+
|      data|tipo_cartao|tipo_compra|sexo|montante|        cidade|
+----------+-----------+-----------+----+--------+--------------+
|2015-03-03|   Platinum|      Bills|   F|  738108|Greater Mumbai|
|2014-03-26|       Gold|      Bills|   F|  234200|         Delhi|
|2014-05-06|     Silver|       Food|   F|  138342|     Bengaluru|
|2014-02-24|       Gold|       Food|   F|  236980|Greater Mumbai|
|2014-12-24|     Silver|       Food|   F|  168546|     Bengaluru|
+----------+-----------+-----------+----+--------+--------------+
only showing top 5 rows



In [103]:
df2.dtypes

[('data', 'date'),
 ('tipo_cartao', 'string'),
 ('tipo_compra', 'string'),
 ('sexo', 'string'),
 ('montante', 'bigint'),
 ('cidade', 'string')]